<a href="https://colab.research.google.com/github/ilsilfverskiold/ai-tech-news-bot/blob/main/computer-vision/cook/image-classification/Image_classification_ConvNeXT_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Image classification with ConvNEXT using a Hugging Face dataset.**

---



The model is set at facebook/convnext-large-224 but could do just as well with facebook/convnext-tiny-224.

Batch size is 32, epoch is 3.

**Make sure you change the dataset to what you need.** My dataset I've used has both a training and a validation set, so change the code accordingly if you don't have a validation set.

In [ ]:
!pip install -q transformers datasets

In [ ]:
dataset = "ilsilfverskiold/traffic-camera-norway-images" # public dataset (possible to import private too)
model = "facebook/convnext-large-224" # decide on your model
learning_rate = 5e-5
epochs = 3

Import the dataset from huggingface below.

In [ ]:
from datasets import load_dataset

dataset = load_dataset(dataset)

dataset

Check the features and get the labels. Make sure the images are in PIL format.

In [ ]:
dataset["train"].features

In [ ]:
dataset["train"][0]

In [ ]:
labels = dataset["train"].features["label"].names
print(labels)

['high-traffic', 'low-traffic', 'medium-traffic', 'no-traffic']


In [ ]:
id2label = {k:v for k,v in enumerate(labels)}
label2id = {v:k for k,v in enumerate(labels)}

Preprocess the dataset for fine-tuning with ViT/ConvNEXT/Swin Transformer we'll use an image prcoessor to normalize. The image processor ensures that every input image conforms to expectations (input image size and pixel value range).

In [ ]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained(model)

The code below is defining a set of image transformations that are applied to the training data. These transformations prepare images for input into a neural network by normalizing them and augmenting the dataset to improve model robustness.

In [ ]:
from torchvision.transforms import (
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

transform = Compose(
    [
     RandomResizedCrop(image_processor.size["shortest_edge"]),
     RandomHorizontalFlip(),
     ToTensor(),
     normalize
    ]
)

def train_transforms(examples):
  examples["pixel_values"] = [transform(image.convert("RGB")) for image in examples["image"]]

  return examples

In [ ]:
processed_dataset = dataset.with_transform(train_transforms)

The purpose of the collate_fn function below is to control how a list of samples (gathered from the dataset) is merged into a single batch. This function is crucial for ensuring that batches are structured properly before being fed into a model during training or evaluation.

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(examples):
  pixel_values = torch.stack([example["pixel_values"] for example in examples])
  labels = torch.tensor([example["label"] for example in examples])

  return {"pixel_values": pixel_values, "labels": labels}

dataloader = DataLoader(processed_dataset["train"], collate_fn=collate_fn, batch_size=4, shuffle=True)
dataloader_validation = DataLoader(processed_dataset["validation"], collate_fn=collate_fn, batch_size=4, shuffle=True)

In [ ]:
import torch

batch = next(iter(dataloader))
for k,v in batch.items():
  print(k,v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


We use the labels we set up earlier from the dataset when importing the pre-trained model below, we also tell it to ignore the pre-defined labels that it previously have been trained on.

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(model,
                                                        id2label=id2label,
                                                        label2id=label2id,
                                                        ignore_mismatched_sizes=True) # set to true to ignore the pre-defined labels

Train the model, make sure you check the training loss/accuracy, you want one to consistently go down while accuracy should go up. If it fluctuates makes sure you check the performance of the model. I'm using other metrics like precision, recall and f1 too but accuracy is usually the most important of them.

In [ ]:
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for epoch in range(epochs):
    print("Epoch:", epoch)
    model.train()
    training_predictions = []
    training_labels = []

    for batch in tqdm(dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()

        outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])
        loss, logits = outputs.loss, outputs.logits
        loss.backward()
        optimizer.step()

        preds = logits.argmax(-1)
        training_predictions.extend(preds.cpu().numpy())
        training_labels.extend(batch["labels"].cpu().numpy())

    # Calculate training metrics (you can remove some of these if you only need accuracy or so)
    train_accuracy = accuracy_score(training_labels, training_predictions)
    train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(training_labels, training_predictions, average='weighted')

    print(f"\nTraining Loss: {loss.item()}")
    print(f"Training Accuracy: {train_accuracy}")
    print(f"Training Precision: {train_precision}")
    print(f"Training Recall: {train_recall}")
    print(f"Training F1 Score: {train_f1}")

    # Evaluate on validation set
    model.eval()
    with torch.no_grad():
        validation_predictions = []
        validation_labels = []

        for batch in tqdm(dataloader_validation):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])
            logits = outputs.logits

            preds = logits.argmax(-1)
            validation_predictions.extend(preds.cpu().numpy())
            validation_labels.extend(batch["labels"].cpu().numpy())

        val_accuracy = accuracy_score(validation_labels, validation_predictions)
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(validation_labels, validation_predictions, average='weighted')

        print(f"\nValidation Accuracy: {val_accuracy}")
        print(f"Validation Precision: {val_precision}")
        print(f"Validation Recall: {val_recall}")
        print(f"Validation F1 Score: {val_f1}\n")

Save the model below so we can du inference on it. Sometimes you can have good metrics but the model doesn't perform well on new data, so check both.

In [ ]:
model.save_pretrained("trained_model")
model.config.save_pretrained("trained_model")

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("trained_model")

In [ ]:
from transformers import pipeline

pipe = pipeline("image-classification", model=model, image_processor=image_processor)

I usually mount my Google Drive to use new images to test with. This is not necessary if you want to test it in Hugging Face after you've deployed it.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image

image_path = '/content/drive/MyDrive/your_image_you_want_to_test.png'

image = Image.open(image_path)
image

results = pipe(image)
results

I also check the validation set and loop through a few of them to see how they do.

In [ ]:
from PIL import Image

for i in range(100):
    image_data = dataset['validation'][i]['image']
    label_index = dataset['validation'][i]['label']

    if not isinstance(image_data, Image.Image):
        image = Image.open(image_data)
    else:
        image = image_data

    results = pipe(image)

    print(f"Results for image {i+1}:")
    print(results)
    print("Actual label:", id2label[label_index])
    print("----------------------------------")

If you're ready to go we can push it to Hugging Face. You'll need a token that has both read/write rights that you find under Settings in your Hugging Face account.

In [ ]:
!huggingface-cli login

In [ ]:
repo_name = ""

model.push_to_hub(repo_name)
image_processor.push_to_hub(repo_name)